In [11]:
!pip install kagglehub torch torchvision tqdm --quiet

import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


Using device: cuda


In [12]:
import kagglehub

path = kagglehub.dataset_download("fedesoriano/cifar100")
print("Path to dataset files:", path)


Using Colab cache for faster access to the 'cifar100' dataset.
Path to dataset files: /kaggle/input/cifar100


In [13]:
# Load CIFAR-100 class names (in torchvision order)
CIFAR100_CLASSES = datasets.CIFAR100(root=".", download=True).classes

superclasses = {
    "aquatic_mammals": ["beaver", "dolphin", "otter", "seal", "whale"],
    "fish": ["aquarium_fish", "flatfish", "ray", "shark", "trout"],
    "flowers": ["orchid", "poppy", "rose", "sunflower", "tulip"],
    "food_containers": ["bottle", "bowl", "can", "cup", "plate"],
    "fruit_and_vegetables": ["apple", "mushroom", "orange", "pear", "sweet_pepper"],
}

chosen_classes = [cls for group in superclasses.values() for cls in group]
chosen_class_indices = [CIFAR100_CLASSES.index(cls) for cls in chosen_classes]

print("Chosen classes (25 total):", chosen_classes)


Chosen classes (25 total): ['beaver', 'dolphin', 'otter', 'seal', 'whale', 'aquarium_fish', 'flatfish', 'ray', 'shark', 'trout', 'orchid', 'poppy', 'rose', 'sunflower', 'tulip', 'bottle', 'bowl', 'can', 'cup', 'plate', 'apple', 'mushroom', 'orange', 'pear', 'sweet_pepper']


In [14]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])

train_full = datasets.CIFAR100(root=".", train=True, download=True, transform=transform)
test_full  = datasets.CIFAR100(root=".", train=False, download=True, transform=transform)

def remap_labels(dataset, allowed_indices):
    """Filter dataset and relabel classes from 0..N-1"""
    mapping = {orig:new for new, orig in enumerate(allowed_indices)}
    imgs, labels = [], []
    for img, lbl in dataset:
        if lbl in allowed_indices:
            imgs.append(img)
            labels.append(mapping[lbl])
    return TensorDataset(torch.stack(imgs), torch.tensor(labels))

# Phase 1 = first 3 classes per superclass (5×3 = 15 classes)
phase1_classes = [cls for group in superclasses.values() for cls in group[:3]]
phase1_indices = [CIFAR100_CLASSES.index(cls) for cls in phase1_classes]

train_phase1 = remap_labels(train_full, phase1_indices)
test_phase1  = remap_labels(test_full, phase1_indices)
train_phase2 = remap_labels(train_full, chosen_class_indices)
test_phase2  = remap_labels(test_full, chosen_class_indices)

print(f"Phase1 → {len(train_phase1)} train / {len(test_phase1)} test")
print(f"Phase2 → {len(train_phase2)} train / {len(test_phase2)} test")


Phase1 → 7500 train / 1500 test
Phase2 → 12500 train / 2500 test


In [15]:
class SimpleCNN(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.pool = nn.MaxPool2d(2,2)
        self.fc1 = nn.Linear(64*8*8, 256)
        self.fc2 = nn.Linear(256, num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 64*8*8)
        x = F.relu(self.fc1(x))
        return self.fc2(x)

print("✅ CNN ready.")


✅ CNN ready.


In [16]:
def train_model(model, dataloader, criterion, optimizer, epochs=5):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for imgs, labels in tqdm(dataloader, desc=f"Epoch {epoch+1}/{epochs}"):
            imgs, labels = imgs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(imgs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch {epoch+1} Loss: {total_loss/len(dataloader):.4f}")

def evaluate_model(model, dataloader):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for imgs, labels in dataloader:
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    return correct / total * 100


In [17]:
train_loader_p1 = DataLoader(train_phase1, batch_size=64, shuffle=True)
test_loader_p1  = DataLoader(test_phase1, batch_size=64, shuffle=False)

model_p1 = SimpleCNN(num_classes=len(phase1_indices)).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_p1.parameters(), lr=0.001)

train_model(model_p1, train_loader_p1, criterion, optimizer, epochs=5)
phase1_acc = evaluate_model(model_p1, test_loader_p1)
print(f"✅ Phase 1 Accuracy: {phase1_acc:.2f}%")


Epoch 1/5: 100%|██████████| 118/118 [00:01<00:00, 81.89it/s]


Epoch 1 Loss: 1.9735


Epoch 2/5: 100%|██████████| 118/118 [00:01<00:00, 96.01it/s]


Epoch 2 Loss: 1.4894


Epoch 3/5: 100%|██████████| 118/118 [00:01<00:00, 98.52it/s]


Epoch 3 Loss: 1.2556


Epoch 4/5: 100%|██████████| 118/118 [00:01<00:00, 61.90it/s]


Epoch 4 Loss: 1.0592


Epoch 5/5: 100%|██████████| 118/118 [00:01<00:00, 59.36it/s]


Epoch 5 Loss: 0.9215
✅ Phase 1 Accuracy: 61.53%


In [20]:
train_loader_p2 = DataLoader(train_phase2, batch_size=64, shuffle=True)
test_loader_p2  = DataLoader(test_phase2, batch_size=64, shuffle=False)

model_p2 = SimpleCNN(num_classes=len(chosen_class_indices)).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_p2.parameters(), lr=0.001)

phase2_acc_list = []
epochs_needed = None

for epoch in range(1, 6):  # up to 15 epochs
    train_model(model_p2, train_loader_p2, criterion, optimizer, epochs=1)
    acc = evaluate_model(model_p2, test_loader_p2)
    phase2_acc_list.append(acc)
    print(f"Epoch {epoch} → Accuracy {acc:.2f}%")
    if acc >= phase1_acc and epochs_needed is None:
        epochs_needed = epoch

final_phase2_acc = phase2_acc_list[-1]
print(f"\n✅ Final Phase 2 Accuracy: {final_phase2_acc:.2f}%")
print(f"📊 Accuracy Difference = {final_phase2_acc - phase1_acc:.2f}%")
if epochs_needed:
    print(f"🕓 Epochs in Phase 2 to reach Phase 1 accuracy: {epochs_needed}")
else:
    print("⚠️ Phase 2 never matched Phase 1 accuracy within 15 epochs.")


Epoch 1/1: 100%|██████████| 196/196 [00:01<00:00, 170.23it/s]


Epoch 1 Loss: 2.3267
Epoch 1 → Accuracy 39.88%


Epoch 1/1: 100%|██████████| 196/196 [00:01<00:00, 181.77it/s]


Epoch 1 Loss: 1.7831
Epoch 2 → Accuracy 46.08%


Epoch 1/1: 100%|██████████| 196/196 [00:01<00:00, 164.71it/s]


Epoch 1 Loss: 1.5163
Epoch 3 → Accuracy 49.12%


Epoch 1/1: 100%|██████████| 196/196 [00:01<00:00, 176.62it/s]


Epoch 1 Loss: 1.3094
Epoch 4 → Accuracy 51.44%


Epoch 1/1: 100%|██████████| 196/196 [00:01<00:00, 186.51it/s]


Epoch 1 Loss: 1.1091
Epoch 5 → Accuracy 50.76%

✅ Final Phase 2 Accuracy: 50.76%
📊 Accuracy Difference = -10.77%
⚠️ Phase 2 never matched Phase 1 accuracy within 15 epochs.


In [19]:
print("===== SUMMARY =====")
print(f"Phase 1 Accuracy: {phase1_acc:.2f}%")
print(f"Final Phase 2 Accuracy: {final_phase2_acc:.2f}%")
print(f"Accuracy Difference: {final_phase2_acc - phase1_acc:.2f}%")
if epochs_needed:
    print(f"Epochs to reach Phase 1 Accuracy in Phase 2: {epochs_needed}")


===== SUMMARY =====
Phase 1 Accuracy: 61.53%
Final Phase 2 Accuracy: 51.48%
Accuracy Difference: -10.05%
